In [241]:
(eval-and-compile
 
  (setv comma '·)
 
  (defn drop-until [x l] 
    (do
      (try 
        (setv y (.index l x))
        (except [ValueError] (setv y -1)))
      (cut l (inc y))))

  (defn take-until [x l] 
    (do
      (try 
        (setv y (.index l x))
        (except [ValueError] (setv y (len l))))
      (cut l 0 y)))

  (defn hcdot [expr] 
    (list 
      (map (fn [x] (if (= x hy.HyCons) comma x)) expr)))
 
  (defn args [expr]
    (if (= (first expr) comma) [] (take-until comma expr))))

(defmacro 𝜆 [&rest expr]
  `(fn [~@(args expr)] ~(first (drop-until comma expr))))

(defsharp L [expr]
  (do
    (print (first expr))
    (setv expr (hcdot expr))
    (print
      expr
      (drop-until comma `~expr) 
      (take-until comma `~expr))
    `(fn ~(first expr) ~(second expr))))

; Ÿ Application
(defmacro Ÿ [func &rest args] (print `(apply ~func ~args)))
; 𝜆 lambda function / expression / abstraction
(defmacro λ [&rest expr] (print expr))

<function <lambda> at 0x000002190AA6DB70>

#### Identity function

Function returns same value that is passed as an argument:

In [242]:
; ((𝜆 x \. x) 1) -> 1
((𝜆 x · x) 1)

1

In [243]:
((𝜆 x · (* x 2)) 2)

4

In [244]:
((𝜆 x · (* x ((𝜆 x · x) 2))) 2)

4

In [245]:
(macroexpand '(𝜆 x · (* x y)))

('fn' ['x'] ('*' 'x' 'y'))

In [246]:
(macroexpand '(𝜆 x · (𝜆 y · (* y x))))

('fn' ['x'] ('𝜆' 'y' '·' ('*' 'y' 'x')))

In [247]:
(((𝜆 x · (𝜆 y · (* y x))) 2) 2)

4

#### Constants

Whatever is the argument, return 1:

In [248]:
((𝜆 x · 2) 1)

2

In [249]:
(, ((𝜆 x · x) 1) ((𝜆 x · x) 2) ((𝜆 x · x) 3))

(1, 2, 3)

#### Infinite recursion

Produce infinite loop:

In [250]:
(try
 ; we will catch recursion error from this line, otherwise output would be messier
 ((𝜆 x · (x x)) (𝜆 x · (x x)))
 (except (e RecursionError)
   (print "Eternal loop achieved!")))

Eternal loop achieved!


#### No arguments?!

In original Lambda Calculus functions must have one and only one argument. Lambda macro in Calchylus is not that restricted. We can omit argument for simplicity:

In [252]:
((𝜆 · 2))

2

#### Many arguments

Also multiple arguments can be passed to the lambda function althought the original specification allows one and only one argument to be passed to the Lambda function.

Support for both zero and multiple arguments simpifies the Lambda expression notation.

In [253]:
((𝜆 x y · (+ x y)) 1 2)

3

In [55]:
;(λ x '. x)

;(Ÿ (fn [x] x) 'a)

#L (x , x)

('x' ',' 'x')
['x', ',', 'x'] ['x', ',', 'x'] ['x', ',', 'x']


In [56]:
; Abstraction: 𝜆x.x -> creates a definition of (fn [x] x) which is the identity function, returns same value as passed
; Application: (𝜆x.x 1) -> calls for identity function with a value 1, so the output / value of this application is 1
; 𝜆x.y -> constant, what ever value x is, y is returned